In [1]:
# Task 1
"""This file shows a couple of implementations of the perceptron learning
algorithm. It is based on the code from Lecture 3, but using the slightly
more compact perceptron formulation that we saw in Lecture 6.

There are two versions: Perceptron, which uses normal NumPy vectors and
matrices, and SparsePerceptron, which uses sparse vectors and matrices.
The latter may be faster when we have high-dimensional feature representations
with a lot of zeros, such as when we are using a "bag of words" representation
of documents.
"""

import numpy as np
from sklearn.base import BaseEstimator
class LinearClassifier(BaseEstimator):
    """
    General class for binary linear classifiers. Implements the predict
    function, which is the same for all binary linear classifiers. There are
    also two utility functions.
    """

    def decision_function(self, X):
        """
        Computes the decision function for the inputs X. The inputs are assumed to be
        stored in a matrix, where each row contains the features for one
        instance.
        """
        return X.dot(self.w)

    def predict(self, X):
        """
        Predicts the outputs for the inputs X. The inputs are assumed to be
        stored in a matrix, where each row contains the features for one
        instance.
        """

        # First compute the output scores
        scores = self.decision_function(X)

        # Select the positive or negative class label, depending on whether
        # the score was positive or negative.
        out = np.select([scores >= 0.0, scores < 0.0],
                        [self.positive_class,
                         self.negative_class])
        return out

    def find_classes(self, Y):
        """
        Finds the set of output classes in the output part Y of the training set.
        If there are exactly two classes, one of them is associated to positive
        classifier scores, the other one to negative scores. If the number of
        classes is not 2, an error is raised.
        """
        classes = sorted(set(Y))
        if len(classes) != 2:
            raise Exception("this does not seem to be a 2-class problem")
        self.positive_class = classes[1]
        self.negative_class = classes[0]

    def encode_outputs(self, Y):
        """
        A helper function that converts all outputs to +1 or -1.
        """
        return np.array([1 if y == self.positive_class else -1 for y in Y])


In [2]:
# Using hinge loss function algorithm
class Pegasos_SVC(LinearClassifier):
    """
    A straightforward implementation of the perceptron learning algorithm.
    """

    def __init__(self, n_iter=1000):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.n_iter = n_iter

    def fit(self, X, Y):
        """
        Train a linear classifier using the perceptron learning algorithm.
        """

        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        self.find_classes(Y)

        # Convert all outputs to +1 (for the positive class) or -1 (negative).
        Ye = self.encode_outputs(Y)

        # If necessary, convert the sparse matrix returned by a vectorizer
        # into a normal NumPy matrix.
        if not isinstance(X, np.ndarray):
            X = X.toarray()

        # Initialize the weight vector to all zeros.
        n_features = X.shape[1]
        self.w = np.zeros(n_features)
        # Pegasos algorithm 1 using hinge loss objective function for the linear SVC classfier. The mehtod defiens the 
        # input and output of the loss function. Also, it defiens the regulaization parameter as r. 
        t=0
        self.r=  1/(n_features)# Regularization(λ)
        for i in range(self.n_iter):
            for x, y in list(zip(X, Ye)):
                t=t+1
                eta = 1/(t*self.r) # learning ratein gradient descent(η)
                # Compute the output score for this instance.
                score = x.dot(self.w)
                # If there was an error, update the weights.
                if y*score <1:
                    self.w= (1- eta*self.r)*(self.w) + (eta*y)*x
                
                else:
                    self.w= (1-eta*self.r)*(self.w)





In [3]:
# using log loss function algorithm
class Pegasos_LR(LinearClassifier):
    """
    A straightforward implementation of the perceptron learning algorithm.
    """

    def __init__(self, n_iter=1000):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.n_iter = n_iter

    def fit(self, X, Y):
        """
        Train a linear classifier using the perceptron learning algorithm.
        """

        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        self.find_classes(Y)

        # Convert all outputs to +1 (for the positive class) or -1 (negative).
        Ye = self.encode_outputs(Y)

        # If necessary, convert the sparse matrix returned by a vectorizer
        # into a normal NumPy matrix.
        if not isinstance(X, np.ndarray):
            X = X.toarray()

        # Initialize the weight vector to all zeros.
        n_features = X.shape[1]
        self.w = np.zeros(n_features)
        t=0
        self.r= 1/(n_features)# Regularization(λ)
        # Pegasos algorithm 2 using log loss objective function:
        for i in range(self.n_iter):
            for x, y in list(zip(X, Ye)):
                t=t+1
                gl= 1/(t*self.r) # learning ratein gradient descent(η)
                # Compute the output score for this instance.
                score = x.dot(self.w)
                # If there was an error, update the weights.
                grad_log_loss= (y/(1+np.exp(y*score)))*x
                
                self.w= (1-gl*self.r)*(self.w) + gl*grad_log_loss
                
               

In [4]:

import time

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

#from aml_perceptron import Perceptron, SparsePerceptron

# This function reads the corpus, returns a list of documents, and a list
# of their corresponding polarity labels. 
def read_data(corpus_file):
    X = []
    Y = []
    with open(corpus_file, encoding='utf-8') as f:
        for line in f:
            _, y, _, x = line.split(maxsplit=3) # For binary classification
            X.append(x.strip())
            Y.append(y)
    return X, Y


if __name__ == '__main__':
    
    # Read all the documents.
    X, Y = read_data(r'C:\Users\Amerm\Desktop\pa2b\data\all_sentiment_shuffled.txt')
    # Split into training and test parts.
    Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2,random_state=0)

    # Set up the preprocessing steps and the classifier.
    pipeline = make_pipeline(
        TfidfVectorizer(),
        SelectKBest(k=1000),
        Normalizer(),

        # NB that this is our Perceptron, not sklearn.linear_model.Perceptron
        Pegasos_SVC() 
       # Pegasos_LR()
    )

    # Train the classifier.
    t0 = time.time()
    pipeline.fit(Xtrain, Ytrain)
    t1 = time.time()
    print('Training time: {:.2f} sec.'.format(t1-t0))

    # Evaluate on the test set.
    Yguess = pipeline.predict(Xtest)
    print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))


Training time: 126.33 sec.
Accuracy: 0.8267.


The accuracy results for the linear classifier SVC was 0.83 and for the logistic classifeir was 0.8 so both of them are 
fullfiling the goal from implemnting these classifier after transling the pseudo code to python program. 

In [ ]:
# Task 2 

In [5]:
import numpy as np
from sklearn.base import BaseEstimator
import random
import scipy as sp

class LinearClassifier(BaseEstimator):
    """
    General class for binary linear classifiers. Implements the predict
    function, which is the same for all binary linear classifiers. There are
    also two utility functions.
    """

    def decision_function(self, X):
        """
        Computes the decision function for the inputs X. The inputs are assumed to be
        stored in a matrix, where each row contains the features for one
        instance.
        """
        return X.dot(self.w)

    def predict(self, X):
        """
        Predicts the outputs for the inputs X. The inputs are assumed to be
        stored in a matrix, where each row contains the features for one
        instance.
        """

        # First compute the output scores
        scores = self.decision_function(X)
        A = []
        for row in scores:
            A.append(np.argmax(row))

        # Select the positive or negative class label, depending on whether
        # the score was positive or negative.
        out = self.decode_multi_outputs(A)
        return out

    def find_classes(self, Y):
        """
        Finds the set of output classes in the output part Y of the training set.
        If there are exactly two classes, one of them is associated to positive
        classifier scores, the other one to negative scores. If the number of
        classes is not 2, an error is raised.
        """
        classes = sorted(set(Y))
        if len(classes) != 2:
            raise Exception("this does not seem to be a 2-class problem")
        self.positive_class = classes[1]
        self.negative_class = classes[0]

    def encode_outputs(self, Y):
        """
        A helper function that converts all outputs to +1 or -1.
        """
        return np.array([1 if y == self.positive_class else -1 for y in Y])

    def find_number_of_classes(self, Y):
        i = 0
        classes = sorted(set(Y))
        for c in classes:
            print(c, "has number", i)
            i += 1
        return len(classes)

    def encode_multi_outputs(self, Y):
        switcher = {
            "books": 0,
            "camera": 1,
            "dvd": 2,
            "health": 3,
            "music": 4,
            "software": 5
        }
        Ye = list(map(switcher.get, Y))
        return Ye

    def decode_multi_outputs(self, Y):
        switcher = {
            0: "books",
            1: "camera",
            2: "dvd",
            3: "health",
            4: "music",
            5: "software"
        }
        Yd = list(map(switcher.get, Y))
        return Yd
##### The following part is for the optional task.

### Sparse and dense vectors don't collaborate very well in NumPy/SciPy.
### Here are two utility functions that help us carry out some vector
### operations that we'll need.

def add_sparse_to_dense(x, w, factor):
    """
    Adds a sparse vector x, scaled by some factor, to a dense vector.
    This can be seen as the equivalent of w += factor * x when x is a dense
    vector.
    """
    w[x.indices] += factor * x.data

def sparse_dense_dot(x, w):
    """
    Computes the dot product between a sparse vector x and a dense vector w.
    """
    return np.dot(w[x.indices], x.data)

In [8]:
class MultiClassSVM(LinearClassifier):
    """
    A straightforward implementation of the perceptron learning algorithm.
    """

    def __init__(self, n_iter=100000):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.n_iter = n_iter

    def fit(self, X, Y):
        """
        Train a linear classifier using the pegasos learning algorithm.
        """

        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        #self.find_classes(Y)
        classNumber = self.find_number_of_classes(Y)
        # Convert all outputs to +1 (for the positive class) or -1 (negative).
        #Ye = self.encode_outputs(Y)
        Ye = self.encode_multi_outputs(Y)
        # If necessary, convert the sparse matrix returned by a vectorizer
        # into a normal NumPy matrix.
        if not isinstance(X, np.ndarray):
            X = X.toarray()

        # Initialize the weight vector to all zeros.
        n_features = X.shape[1]
        self.w = np.zeros((n_features, classNumber))
        
        self.r=1/(n_features)
        
        XY = list(zip(X, Ye))

        for i in range(self.n_iter):
            x, y = random.choice(XY)
            t = i + 1
            eta = 1/(t*self.r)
            # Compute the output score for this instance.
            scoreYi = x.dot(self.w[:, y])
            A = []
            for j in range(classNumber):
                scoreY = x.dot(self.w[:, j])
                if y == j:
                    A.append(0 - scoreYi + scoreY)
                else:
                    A.append(1 - scoreYi + scoreY)
            yHat = np.argmax(A)
            m1 = np.zeros((n_features, classNumber))
            m2 = np.zeros((n_features, classNumber))
            m1[:, yHat] = x
            m2[:, y] = x
            self.w = (1 - eta * self.r) * self.w - eta * (m1 - m2)



class MultiClassLR(LinearClassifier):
    """
    A straightforward implementation of the perceptron learning algorithm.
    """

    def __init__(self, n_iter=100000):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.n_iter = n_iter

    def fit(self, X, Y):
        """
        Train a linear classifier using the pegasos learning algorithm.
        """
        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        #self.find_classes(Y)
        classNumber = self.find_number_of_classes(Y)
        # Convert all outputs to +1 (for the positive class) or -1 (negative).
        #Ye = self.encode_outputs(Y)
        Ye = self.encode_multi_outputs(Y)
        # If necessary, convert the sparse matrix returned by a vectorizer
        # into a normal NumPy matrix.
        if not isinstance(X, np.ndarray):
            X = X.toarray()

        # Initialize the weight vector to all zeros.
        n_features = X.shape[1]
        self.w = np.zeros((n_features, classNumber))
        self.r= 1/(n_features)
        t=0
        XY = list(zip(X, Ye))

        for i in range(self.n_iter):
            x, y = random.choice(XY)
            t = t + 1
            eta = 1/(t*self.r)
            # Compute the output score for this instance.
            scores = x.dot(self.w)
            grad_log_loss = np.zeros((n_features, classNumber))
            p = sp.special.softmax(scores)
            p12 = np.zeros((n_features, classNumber))
            p12[:, y] = x
            for r in range(classNumber):
                p34 = np.zeros((n_features, classNumber))
                p34[:, r] = x
                grad_log_loss += p * p34 - p12
                
            self.w = (1 - eta * self.r) * self.w - eta * grad_log_loss

In [10]:
import time

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

#from aml_perceptron import Perceptron, SparsePerceptron

# This function reads the corpus, returns a list of documents, and a list
# of their corresponding polarity labels. 
def read_data(corpus_file):
    X = []
    Y = []
    with open(corpus_file, encoding='utf-8') as f:
        for line in f:
            #_, y, _, x = line.split(maxsplit=3)
            y, _, _, x = line.split(maxsplit=3) # For multi-class classification
            X.append(x.strip())
            Y.append(y)
    return X, Y


if __name__ == '__main__':
    
    # Read all the documents.
    X, Y = read_data(r'C:\Users\Amerm\Desktop\pa2b\data\all_sentiment_shuffled.txt')
    # Split into training and test parts.
    Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2,random_state=0)

    # Set up the preprocessing steps and the classifier.
    pipeline = make_pipeline(
        TfidfVectorizer(),
        SelectKBest(k=1000),
        Normalizer(),
        #MultiClassLR()
        # NB that this is our Perceptron, not sklearn.linear_model.Perceptron
        MultiClassSVM()
        #OneVsRestClassifier(PegasosLog())
        #OneVsRestClassifier(PegasosHinge())
        #OneVsOneClassifier(PegasosLog())
        #OneVsOneClassifier(PegasosHinge())
        #Pegasos_SVC() 
        
    )

    # Train the classifier.
    t0 = time.time()
    pipeline.fit(Xtrain, Ytrain)
    t1 = time.time()
    print('Training time: {:.2f} sec.'.format(t1-t0))

    # Evaluate on the test set.
    Yguess = pipeline.predict(Xtest)
    print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))


books has number 0
camera has number 1
dvd has number 2
health has number 3
music has number 4
software has number 5
Training time: 12.28 sec.
Accuracy: 0.9287.
